# Order price modification
## 1. Single order

In [1]:
MARKET = 'NSE'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    pd.options.display.float_format = '{:,.2f}'.format # set float precision with comma
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

### Experimenting on PAPER

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PAPER, ibp.MASTERCID

In [6]:
# * IMPORTS
from ib_insync import IB, Stock, Order, LimitOrder
from support import get_prec
from typing import Sequence, List
from tqdm import tqdm

### Let order a symbol
* (say DIS)

In [7]:
SYMBOL = 'RELIANCE'
EXCHANGE = 'NSE'
CURRENCY = 'INR'

QTY = 200

In [8]:
# Set the expected price
df_unds = pd.read_pickle(DATAPATH.joinpath('df_unds.pkl'))
price = df_unds[df_unds.symbol == SYMBOL].undPrice.iloc[0]
expPrice = get_prec(price * 1.2, 0.05)

In [15]:
# * REPRICE ORDERS UPON EXECUTION

def onExecDetails(trade, fill, SCALE=0.5):
    
    # get the symbol of the trade filled
    try:
        symbol = {t.contract.symbol for t in trade}
    except TypeError:
        symbol = {trade.conctract.symbol}
    
    # Download orders and get trades
    ib.reqOpenOrders() 
    trades = ib.trades()
    
    # * TARGET DATAFRAME
    
    # . make the df
    df = util.df(t.contract for t in trades).iloc[:, :6]\
                .assign(contract=[t.contract for t in trades],
                        order=[t.order for t in trades],
                        status=[t.orderStatus.status for t in trades],
                        )\
                .join(util.df(t.order 
                              for t in trades).iloc[:, 2:7])\
                .rename(columns={'lastTradeDateOrContractMonth': 'expiry'})
    
    # . filter the df
    ACTIVE_STATUS = ['ApiPending', 'PendingSubmit', 'PreSubmitted', 'Submitted']
    mask = df.status.isin(ACTIVE_STATUS) & (df.action == "SELL") & (df.symbol.isin(symbol))
    df = df[mask]
    
    # . set the new price
    df['newLmt'] = np.where(df.action == 'SELL', 
                            df.lmtPrice + df.lmtPrice*(1+SCALE), 
                            df.lmtPrice - df.lmtPrice*(1-SCALE))
    
    df['newLmt'] = df['newLmt'].apply(lambda x: get_prec(x, ibp.PREC))    
    
    # change order price to new limit price
    for i in df.itertuples():
        i.order.lmtPrice = i.newLmt
        
    # build the contract, orders
    cos = tuple(zip(df.contract, df.order))
        
    modified_trades = place_orders(ib = ib, cos = cos)

In [16]:
# * SET THE CONNECTION
try:
    ib.isConnected()
except NameError:
    ib = IB().connect(HOST, PORT, CID)

In [17]:
ct = ib.qualifyContracts(Stock(symbol = SYMBOL, exchange = EXCHANGE, currency = CURRENCY))[0]
o = LimitOrder(totalQuantity = QTY, action='SELL', lmtPrice = expPrice)

# Wait for the symbol to execute
ib.execDetailsEvent += onExecDetails

# place the order
ib.placeOrder(ct, o)

Trade(contract=Stock(conId=44652000, symbol='RELIANCE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='RELIANCE', tradingClass='RELIANCE'), order=LimitOrder(orderId=38, clientId=10, action='SELL', totalQuantity=200, lmtPrice=2408.4), orderStatus=OrderStatus(orderId=38, status='PendingSubmit', filled=0, remaining=0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 12, 14, 10, 0, 30, 578331, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

Error 201, reqId 38: Order rejected - reason:Exchange is closed.
Canceled order: Trade(contract=Stock(conId=44652000, symbol='RELIANCE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='RELIANCE', tradingClass='RELIANCE'), order=LimitOrder(orderId=38, clientId=10, permId=1629924785, action='SELL', totalQuantity=200, lmtPrice=2408.4), orderStatus=OrderStatus(orderId=38, status='Cancelled', filled=0.0, remaining=200.0, avgFillPrice=0.0, permId=1629924785, parentId=0, lastFillPrice=0.0, clientId=10, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 12, 14, 10, 0, 30, 578331, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2020, 12, 14, 10, 0, 30, 832998, tzinfo=datetime.timezone.utc), status='Inactive', message=''), TradeLogEntry(time=datetime.datetime(2020, 12, 14, 10, 0, 30, 834993, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 201, reqId 38: Order reject

In [ ]:
# Wait for the symbol to execute
ib.execDetailsEvent += onExecDetails

In [18]:
pd.read_pickle(DATAPATH.joinpath('ztrade.pkl'))

EOFError: Ran out of input

In [20]:
import pickle
with open(DATAPATH.joinpath('ztrade.pkl'), 'rb') as f:
        the_trade = pickle.load(f)

EOFError: Ran out of input

In [ ]:
with IB().connect(HOST, PORT, CID) as ib:
    ib.reqOpenOrders()
    trades = ib.trades()
    openorders = ib.openOrders()

In [ ]:
trades

### Let us check the order

In [ ]:
with IB().connect(HOST, PORT, CID) as ib:
    ib.reqOpenOrders() # Download all orders - once
    
    trades = ib.trades()
    openorders = ib.openOrders()
    
    ct = ib.qualifyContracts(Stock(symbol = SYMBOL, exchange = EXCHANGE, currency = CURRENCY))[0]

In [ ]:
openorders

### Let us modify the order price

In [ ]:
PERMID = 2100649087
m_o = [o for o in openorders if o.permId == PERMID][0] # get the order

# modify the order price
m_o.lmtPrice = 115

In [ ]:
m_o # modified order

In [ ]:
with IB().connect(HOST, PORT, CID) as ib:
    p = ib.placeOrder(ct, m_o)

In [ ]:
p

### Let us check if the price has been modified

In [ ]:
with IB().connect(HOST, PORT, CID) as ib:
    ib.reqOpenOrders()
    trades = ib.trades()
    openorders = ib.openOrders()

In [ ]:
next(o for o in openorders if o.permId==PERMID).lmtPrice

## 2. Modifying multiple orders

### Prepare contracts and orders

In [ ]:
from engine import get_prices

In [ ]:
# * INPUTS
SYMBOLS = ['INTC', 'SBUX', 'IBM', 'MSFT']

In [ ]:
%%time
# * CODE

# Get contracts and prices
with IB().connect(HOST, PORT, CID) as ib:
    cts = ib.qualifyContracts(*[Stock(s, exchange=EXCHANGE, currency=CURRENCY) for s in SYMBOLS])
df_pr = get_prices(cts = cts, MARKET=MARKET, RUN_ON_PAPER=True, FILL_DELAY=5)

In [ ]:
df_pr.price

### Place orders in blocks

In [ ]:
def place_orders(ib: IB, cos: Union[tuple, list], blk_size: int=25) -> List:
    """!!!CAUTION!!!: This places orders in the system
    NOTE: cos could be a single (contract, order) 
          or a tuple/list of ((c1, o1), (c2, o2)...)
          made using tuple(zip(cts, ords))"""
    
    trades = []
    
    if isinstance(cos, (tuple, list)) and (len(cos) == 2):
        c, o = cos
        trades.append(ib.placeOrder(c, o))
        
    else:    
        cobs = {cos[i: i+blk_size] for i in range(0, len(cos), blk_size)}

        for b in tqdm(cobs):
            for c, o in b:
                td = ib.placeOrder(c, o)
                trades.append(td)
            ib.sleep(0.75)
        
    return trades

In [ ]:
ords = [LimitOrder(totalQuantity = 100, action='BUY', lmtPrice = expPrice) 
            for expPrice 
                in [get_prec(p, 0.01) for p in df_pr.price*0.8]]

In [ ]:
# cos = cts[0], ords[0]
cos = tuple(zip(cts, ords))

In [ ]:
placed

### Retrieve the orders

In [ ]:
with IB().connect(HOST, PORT, CID) as ib:
    ib.reqOpenOrders() # Download all orders - once
    trades = ib.trades()

In [ ]:
df = util.df(t.contract for t in trades).iloc[:, :6]\
            .assign(contract=[t.contract for t in trades],
                    order=[t.order for t in trades],
                    status=[t.orderStatus.status for t in trades],
                    )\
            .join(util.df(t.order 
                          for t in trades).iloc[:, 2:7])\
            .rename(columns={'lastTradeDateOrContractMonth': 'expiry'})

In [ ]:
# Filter those in active status
df[df.status.isin(ibp.ACTIVE_STATUS)]

In [ ]:
# * IMPORTS
from engine import Vars
from support import get_prec

In [ ]:
def naked_adjust(SYMBOLS: Union[list, tuple, set], # Filled symbols,
                 MARKET: str='',
                 ACTION_FILTER: str='SELL', # Which options to be targetted
                 SCALE: int=0.5, # Scale up for 'SELL' / down for 'BUY'
                 PLACE_ORDERS: bool = False, 
                ): 
    """Adjusts naked prices to get better deal / prevent too many fills"""
    
    # . Filter active status that can be modified with ACTION_FILTER provided
    ibp = Vars(MARKET) # Dummy - needed only to get ACTIVE_STATUS
    
    with IB().connect(ibp.HOST, ibp.PORT, ibp.MASTERCID) as ib:
        ib.reqOpenOrders() # Download all orders - once
        trades = ib.trades()
        ib.disconnect()
    
    # * TARGET DATAFRAME
    df = util.df(t.contract for t in trades).iloc[:, :6]\
                .assign(contract=[t.contract for t in trades],
                        order=[t.order for t in trades],
                        status=[t.orderStatus.status for t in trades],
                        )\
                .join(util.df(t.order 
                              for t in trades).iloc[:, 2:7])\
                .rename(columns={'lastTradeDateOrContractMonth': 'expiry'})
    

    
    mask = df.status.isin(ibp.ACTIVE_STATUS) & (df.action == ACTION_FILTER)
    df = df[mask]
    
    df['newLmt'] = np.where(df.action == 'SELL', 
                            df.lmtPrice + df.lmtPrice*(1+SCALE), 
                            df.lmtPrice - df.lmtPrice*(1-SCALE))
    
    df['newLmt'] = df['newLmt'].apply(lambda x: get_prec(x, ibp.PREC))
    
#     # . Modified order placement
#     if PLACE_ORDERS:
#         for o in df.orders:
#             o.lmtPrice 
    
    return df

In [ ]:
df = naked_adjust(MARKET=MARKET, SYMBOLS={'MSFT', 'INTC'}, ACTION_FILTER = 'BUY')

In [ ]:
x